In [2]:
# !pip install torch transformers peft bitsandbytes accelerate datasets
# !pip install -U bitsandbytes
# !pip install --upgrade huggingface_hub
# !pip install -U bitsandbytes accelerate transformers
# !pip install trl

In [3]:
from huggingface_hub import login
login()

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import bitsandbytes as bnb
import warnings
warnings.filterwarnings("ignore")

In [6]:
import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForSeq2SeqLM

base_model_name = "google/flan-t5-small"

# Load model
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    base_model_name,
    device_map="auto",
    trust_remote_code=True
)

# Load tokenizer
base_tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)




In [7]:
base_tokenizer.pad_token = base_tokenizer.eos_token
base_tokenizer.padding_side = "right"

In [8]:
train_dataset = load_dataset("imdb", split="train")
test_dataset = load_dataset("imdb", split="test")

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    save_strategy="epoch",
    per_device_train_batch_size=4,
    push_to_hub=True,
    learning_rate=2e-4
)

In [10]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [11]:
def prompt_instruction_function(sample):
  return f"""
    ### Instruction: Use the task below and then summarize the data to provide an output.

    ### Task: Summarize the input and determine whether the input is a positive or negative

    ### Input: {sample["text"]}

    ### Output: {"Negative" if sample["label"] == 0 else "Positive"}
  """

In [12]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=base_model,
    args=training_args,
    train_dataset=train_dataset,
    peft_config=peft_config,
    formatting_func=prompt_instruction_function
  )

In [ ]:
trainer.train()

In [31]:
new_model = "./Fine-Tuned-Google-Flan-t5-small"

In [32]:
trainer.model.save_pretrained(new_model, safe_serialization=True)

In [33]:
base_tokenizer.save_pretrained(new_model)

('./Fine-Tuned-Google-Flan-t5-small/tokenizer_config.json',
 './Fine-Tuned-Google-Flan-t5-small/special_tokens_map.json',
 './Fine-Tuned-Google-Flan-t5-small/spiece.model',
 './Fine-Tuned-Google-Flan-t5-small/added_tokens.json',
 './Fine-Tuned-Google-Flan-t5-small/tokenizer.json')

In [34]:
from peft import PeftModel

merged_model = PeftModel.from_pretrained(
    base_model, new_model
)
merged_model = merged_model.merge_and_unload()

In [35]:
merged_model.save_pretrained(new_model, safe_serialization=True)


In [36]:
base_tokenizer.save_pretrained(new_model)


('./Fine-Tuned-Google-Flan-t5-small/tokenizer_config.json',
 './Fine-Tuned-Google-Flan-t5-small/special_tokens_map.json',
 './Fine-Tuned-Google-Flan-t5-small/spiece.model',
 './Fine-Tuned-Google-Flan-t5-small/added_tokens.json',
 './Fine-Tuned-Google-Flan-t5-small/tokenizer.json')

In [ ]:
base_tokenizer.push_to_hub(f"your-username/model-name")

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...d-Google-Flan-t5-small/spiece.model: 100%|##########|  792kB /  792kB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/atharva77/Fine-Tuned-Google-Flan-t5-small/commit/dd52a59530faf06a156d0897fa6c659b7bee92e4', commit_message='Upload tokenizer', commit_description='', oid='dd52a59530faf06a156d0897fa6c659b7bee92e4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/atharva77/Fine-Tuned-Google-Flan-t5-small', endpoint='https://huggingface.co', repo_type='model', repo_id='atharva77/Fine-Tuned-Google-Flan-t5-small'), pr_revision=None, pr_num=None)

In [ ]:
from huggingface_hub import upload_folder

upload_folder(
    repo_id="your-username/model-name",  # your repo name
    folder_path="./Fine-Tuned-Google-Flan-t5-small",                   # path in Colab
    repo_type="model"
)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...d-Google-Flan-t5-small/spiece.model: 100%|##########|  792kB /  792kB            

  ...-t5-small/adapter_model.safetensors: 100%|##########| 11.0MB / 11.0MB            

  ...gle-Flan-t5-small/model.safetensors:   0%|          | 19.7kB /  308MB            

CommitInfo(commit_url='https://huggingface.co/atharva77/Fine-Tuned-Google-Flan-t5-small/commit/d0b4ead46ab90f858b5ec038e4e11e31609d7995', commit_message='Upload folder using huggingface_hub', commit_description='', oid='d0b4ead46ab90f858b5ec038e4e11e31609d7995', pr_url=None, repo_url=RepoUrl('https://huggingface.co/atharva77/Fine-Tuned-Google-Flan-t5-small', endpoint='https://huggingface.co', repo_type='model', repo_id='atharva77/Fine-Tuned-Google-Flan-t5-small'), pr_revision=None, pr_num=None)

In [5]:
import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForSeq2SeqLM

new_model_name = "atharva77/Fine-Tuned-Google-Flan-t5-small"

# Load model
new_model = AutoModelForSeq2SeqLM.from_pretrained(
    new_model_name,
    device_map="auto",
    trust_remote_code=True
)

# Load tokenizer
new_tokenizer = AutoTokenizer.from_pretrained(new_model_name, use_fast=True)




In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
new_model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): lora.Linear(
                (base_layer): Linear(in_features=512, out_features=384, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=512, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=384, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k): Linear(in_featu

In [25]:
from random import randrange
sample = test_dataset[randrange(len(test_dataset))]
print(sample["text"])
print(sample["label"])
input_text = f"""
      ### Task: Summarize the input and determine whether the input is a positive or negative
      ### Input: I hate the movie.

"""
input = new_tokenizer(input_text, return_tensors="pt").to(device)

output = new_model.generate(**input, max_length=100)

print(new_tokenizer.decode(output[0], skip_special_token=True))


I first played this around 98' or 99' when I was with my friends.I thought the game was really great,and loved it.<br /><br />The game is simple.On one player mode,you go around as James Bond and complete missions in different places like an Arctic wasteland or a city.My favorite was one with a tank.On two player mode,you and a friend choose from any character you wish and go all out with a fight.Through out the area you are in,you will find ammo and weapons to help.From hand guns to rifles to lasers and even your fists work.<br /><br />Again with player two mode,there are lots of places to go,and some to unlock.I find this game really fun,but also very suspenseful.Because,you never really know where your opponent is,and it's surprising to see them behind a door where you are going.<br /><br />This game gets ****(1/2) stars or of ***** Very good!Go play it sometime!
1
<pad> ### Task: Summarize the input and determine whether the input is a positive or negative ### Input: I hate the mov